# core

> Functionalities to search, and retrieve data from pubmed

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from Bio import Entrez
import sys
import pandas as pd
import xlsxwriter
from datetime import datetime, timedelta, date
from collections import defaultdict, Counter
import  pickle
from fastcore.all import *


In [ ]:
#| export
from pubmed_lib.data import *
# from pubmed_lib.parser import *

[autoreload of pubmed_lib.parser failed: Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/Volumes/Users/matu/Documents/Xcode/pubmed_lib/pubmed_lib

In [ ]:
#| export
from dotenv import load_dotenv

load_dotenv('pass.env')
email = os.environ.get('EMAIL')
api_key = os.environ.get('API_KEY')

In [ ]:
#| export 
def searchpb(
    search_term:str, #Query to be search in pubmed
    search_tag:str = 'Author', #Tag to specifiy the search, can be any from pubmed, Defaul: Author
    retmax:int = 5000, #Maximum number of results to be retrieved
    retmode:str ='xml', #Format of the returned data, options are xml, 
    sort:str='relevance', #Way to sort the results
    mindate:int = None, #Initial data to be search from, year
    maxdate:int = None, #Final data to be search from, year
    ):
    """
    It receive a query to be searched in pubmed and return the handler of the search
    """
    Entrez.email = email
    Entrez.api_key = api_key
    if search_tag not in SEARCH_TAGS.keys():
        print(f"Search tag need to be one of the following {SEARCH_TAGS.keys()}")
        raise AttributeError()
    handle = Entrez.esearch(db='pubmed',
                            sort=sort,
                            retmax=retmax,
                            retmode=retmode,
                            term=search_term+SEARCH_TAGS[search_tag],
                            mindate = mindate,
                            maxdate = maxdate)
    return Entrez.read(handle)

In [ ]:
#| export
def fetch_details(
    IdList:list #List of pubmedIDs to fetch the details
    ):
    """
    It receive a list of pubmedIds from a search, and retrieve all the details of those publications
    """
    ids = ','.join(IdList)
    Entrez.email = email
    Entrez.api_key = api_key
    handle = Entrez.efetch(db='pubmed',
                           retmode='xml',
                           id=ids)
    results = Entrez.read(handle)
    return results

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

[autoreload of pubmed_lib.parser failed: Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/Volumes/Users/matu/Documents/Xcode/pubmed_lib/pubmed_lib